In [1]:
from mapknowledge import KnowledgeStore

sckan_version = 'sckan-2025-05-02'
store = KnowledgeStore(sckan_version=sckan_version)


2025-05-21 11:13:42 [info     ] Map Knowledge version 0.21.0 with no cache type=knowledge
2025-05-21 11:13:42 [warning  ] Undefined SCICRUNCH_API_KEY: SciCrunch knowledge will not be looked up type=knowledge
2025-05-21 11:13:57 [info     ] Using knowledge source: sckan-2025-05-02-npo type=knowledge


In [2]:
from pprint import pprint

class SCKANNerves:
    def __init__(self, store):
        self.__store = store

        self.__node_nerve_by_path = {
            path: self.__get_path_node_nerves(path)
            for path in self.__store.connectivity_paths()
        }

        self.__path_by_node_nerve = {
            node_nerve: [path for path, nodes in self.__node_nerve_by_path.items() if node_nerve in nodes]
            for node_nerve in {n for nodes in self.__node_nerve_by_path.values() for n in nodes}
        }

        self.__nerve_by_path = {
            path: [
                nerve
                for node in nodes
                for nerve in (node[0], *node[1])
                if self.is_nerve(nerve)
            ]
            for path, nodes in self.__node_nerve_by_path.items()
        }

        self.__path_by_nerve = {
            nerve: [path for path, nerves in self.__nerve_by_path.items() if nerve in nerves]
            for nerve in {n for nerves in self.__nerve_by_path.values() for n in nerves}
        }

    def __get_path_node_nerves(self, path):
        return self.__store.entity_knowledge(path).get('nerves', [])

    def is_nerve(self, term):
        return self.__store.entity_knowledge(term).get('type') == 'nerve'

    @property
    def nerves(self):
        return {
            nerve: self.__store.entity_knowledge(nerve).get('label')
            for nerve in self.__path_by_nerve
        }

    @property
    def node_nerves(self):
        return list(self.__path_by_node_nerve)

    @property
    def path_nerves(self):
        return self.__nerve_by_path

    def get_nerves(self, path):
        return self.__nerve_by_path.get(path, [])

    def get_node_nerves(self, path):
        return self.__node_nerve_by_path.get(path, [])

    def get_path(self, nerve: str | tuple):
        if isinstance(nerve, str):
            return self.__path_by_nerve.get(nerve)
        elif isinstance(nerve, tuple):
            return self.__path_by_node_nerve.get(nerve)
        return None
    
    
sckan_nerves = SCKANNerves(store)

2025-05-21 11:13:57 [info     ] Consulting NPO for knowledge about ilxtr:neuron-type-keast-4 type=knowledge
2025-05-21 11:13:57 [info     ] Consulting NPO for knowledge about ilxtr:neuron-type-keast-19 type=knowledge
2025-05-21 11:13:57 [info     ] Consulting NPO for knowledge about ilxtr:neuron-type-keast-13 type=knowledge
2025-05-21 11:13:57 [info     ] Consulting NPO for knowledge about ilxtr:neuron-type-keast-17 type=knowledge
2025-05-21 11:13:57 [info     ] Consulting NPO for knowledge about ilxtr:neuron-type-keast-18 type=knowledge
2025-05-21 11:13:57 [info     ] Consulting NPO for knowledge about ilxtr:neuron-type-keast-16 type=knowledge
2025-05-21 11:13:57 [info     ] Consulting NPO for knowledge about ilxtr:neuron-type-keast-11 type=knowledge
2025-05-21 11:13:57 [info     ] Consulting NPO for knowledge about ilxtr:neuron-type-keast-10 type=knowledge
2025-05-21 11:13:57 [info     ] Consulting NPO for knowledge about ilxtr:neuron-type-keast-12 type=knowledge
2025-05-21 11:13:57 

In [3]:
sckan_nerves.nerves

{'ILX:0793822': 'Superior ovarian nerve',
 'ILX:0738312': 'Aortic arch depressor nerve',
 'UBERON:0001649': 'glossopharyngeal nerve',
 'UBERON:0001780': 'spinal nerve',
 'UBERON:0035207': 'deep fibular nerve',
 'UBERON:0022302': 'short ciliary nerve',
 'ILX:0793213': 'white communicating ramus of seventh thoracic spinal nerve',
 'UBERON:0001021': 'nerve',
 'UBERON:0001267': 'femoral nerve',
 'UBERON:0001650': 'hypoglossal nerve',
 'UBERON:0008810': 'nasopalatine nerve',
 'ILX:0785238': 'Anterior ramus of second sacral nerve',
 'ILX:0785733': 'gray communicating ramus of second lumbar nerve',
 'ILX:0738293': 'Ganglioglomerular nerve',
 'ILX:0739303': 'gray communicating ramus of second thoracic nerve',
 'ILX:0793826': 'anterior abdominal vagal trunk',
 'ILX:0794476': 'abdominal branch of vagus nerve',
 'ILX:0793723': 'Auriculotemporal nerve',
 'UBERON:0001492': 'radial nerve',
 'ILX:0793632': 'lumbar colonic nerve',
 'ILX:0793714': 'Mesenteric nerve',
 'ILX:0738372': 'white communicatin

In [4]:
sckan_nerves.path_nerves

{'ilxtr:neuron-type-keast-4': ['ILX:0793559',
  'ILX:0793228',
  'UBERON:0018675',
  'ILX:0739299'],
 'ilxtr:neuron-type-keast-19': [],
 'ilxtr:neuron-type-keast-13': [],
 'ilxtr:neuron-type-keast-17': [],
 'ilxtr:neuron-type-keast-18': [],
 'ilxtr:neuron-type-keast-16': [],
 'ilxtr:neuron-type-keast-11': ['UBERON:0018683', 'ILX:0793559'],
 'ilxtr:neuron-type-keast-10': ['UBERON:0018675', 'ILX:0793559'],
 'ilxtr:neuron-type-keast-12': ['UBERON:0011390'],
 'ilxtr:neuron-type-keast-14': [],
 'ilxtr:neuron-type-keast-3': ['ILX:0793559'],
 'ilxtr:neuron-type-keast-9': ['UBERON:0011390'],
 'ilxtr:neuron-type-keast-8': ['ILX:0793221', 'ILX:0793220'],
 'ilxtr:neuron-type-keast-6': ['ILX:0793220',
  'ILX:0793221',
  'UBERON:0018683',
  'ILX:0785733',
  'ILX:0785825'],
 'ilxtr:neuron-type-keast-7': ['ILX:0793220',
  'ILX:0793221',
  'ILX:0785733',
  'UBERON:0018683',
  'ILX:0785825'],
 'ilxtr:neuron-type-keast-5': ['UBERON:0018675'],
 'ilxtr:neuron-type-keast-20': [],
 'ilxtr:neuron-type-keast-

In [5]:
sckan_nerves.get_nerves('ilxtr:neuron-type-keast-8')

['ILX:0793221', 'ILX:0793220']

In [6]:
sckan_nerves.get_path('ILX:0793221')

['ilxtr:neuron-type-keast-8',
 'ilxtr:neuron-type-keast-6',
 'ilxtr:neuron-type-keast-7',
 'ilxtr:sparc-nlp/swglnd/162',
 'ilxtr:sparc-nlp/kidney/146']